<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" width="300">
<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_dark.png" width="300">

---

# **Análise de Fundos Imobiliarios e criação de dashboard para tomada de decisão de investimento**

Este projeto é uma ferramenta desenvolvida em Python para extrair, manipular e visualizar dados de fundos imobiliários cadastrados na Bolsa de Valores Brasileira (B3). O objetivo é fornecer uma análise detalhada das informações mais relevantes sobre esses fundos, ajudando investidores a tomar decisões mais informadas.

## Funcionalidades

### Extração de Dados
- Coleta informações de todos os fundos imobiliários listados na B3, incluindo dados atualizados sobre:
  - **Setor**
  - **Liquidez Diária (R$)**
  - **P/VP**
  - **Dividend Yield**
  - **Variação de Preço**
  - **Patrimônio Líquido**
  - **Número de Cotistas**

### Manipulação de Dados
- Processa e organiza os dados para facilitar a análise e a visualização.

### Dashboard Interativo
- Cria um painel visual com as seguintes métricas:
  - **Setor**
  - **Liquidez Diária (R$)**
  - **P/VP**
  - **Dividend Yield**
  - **Variação de Preço**
  - **Patrimônio Líquido**
  - **Número de Cotistas**

### Gráficos Dinâmicos
- Inclui gráficos interativos que permitem filtrar e explorar os dados com base em:
  - **Dividend Yield**
  - **Liquidez Diária**
  - **Número de Cotistas**


In [ ]:
import pandas as pd
from io import StringIO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import matplotlib.pyplot as plt
import vizro.plotly.express as px
from vizro import Vizro
import vizro.models as vm

import yfinance as yf

# **Iniciar o crawling**

Vamos utilizar este site: https://www.fundsexplorer.com.br/ranking

**Extração da tabela original**

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.fundsexplorer.com.br/ranking')
driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table')
tb_din = driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table').get_attribute('outerHTML')
html_io = StringIO(tb_din)
driver.quit()

In [ ]:
fiis = pd.read_html(html_io, decimal = ',', thousands = '.')[0]
fiis

In [ ]:
print(f"Total de {fiis.shape[0]} Fundos Imobiliarios")

**Manipulação dos dados**

In [ ]:
fiis = fiis.replace({',':'.', '%':''}, regex = True)
fiis[['Dividend Yield', 'Variação Preço']] = fiis[['Dividend Yield', 'Variação Preço']].apply(pd.to_numeric)
fiis = fiis.set_index('Fundos')
fiis.fillna(0, inplace = True)
fiis['FII'] = fiis.index
fiis['Setor'] = fiis['Setor'].astype(str)
fiis = fiis[fiis['Num. Cotistas'] > 100]
fiis

# **Criação do Dashboard de FIIs**

In [ ]:
page = vm.Page(
    title = 'Dashboard de FIIs',
    components = [
        vm.Graph(id = 'scatter_dy', figure = px.scatter(fiis, x ='Num. Cotistas' , y ='Dividend Yield', color = 'Setor')),
        vm.Graph(id = 'bar_pvp', figure = px.bar(fiis, x ='FII' , y ='P/VP', color = 'Setor')),
        vm.Graph(id = 'bar_patliq', figure = px.bar(fiis, x ='FII' , y ='Patrimônio Líquido', color = 'Setor')),
        
        
    ],
    
    controls = [
        vm.Filter(column = 'Setor'),
        vm.Filter(column = 'Liquidez Diária (R$)', selector = vm.RangeSlider()),
        vm.Filter(column = 'Dividend Yield', selector = vm.RangeSlider()),
        vm.Filter(column = 'Num. Cotistas', selector = vm.RangeSlider())
        
        
    ]
    

)

dashboard = vm.Dashboard(pages = [page])

Vizro().build(dashboard).run()